# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity


In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
# ваш код

In [5]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [6]:
# standard tokenizer

In [7]:
st_vectorizer = CountVectorizer()

In [8]:
X_st = st_vectorizer.fit_transform(train.comment)
X_st_test = st_vectorizer.transform(test.comment) 

In [9]:
X_st.shape, X_st_test.shape

((12970, 63875), (1442, 63875))

In [10]:
# razdel tokenizer

In [11]:
from razdel import tokenize as razdel_tokenize

def raz_tokenizer(text):
    return [token.text for token in list(razdel_tokenize(text))]

In [12]:
raz_vectorizer = CountVectorizer(tokenizer=raz_tokenizer)

In [13]:
X_raz = raz_vectorizer.fit_transform(train.comment)
X_raz_test = raz_vectorizer.transform(test.comment) 

D:\Users\nikol\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
X_raz.shape, X_raz_test.shape

((12970, 64774), (1442, 64774))

In [15]:
y = train.toxic.values
y_test = test.toxic.values

In [16]:
# classifier

In [17]:
clf = MultinomialNB(alpha=1.)

In [18]:
clf.fit(X_st, y)

MultinomialNB()

In [19]:
st_preds = clf.predict(X_st_test)

In [20]:
clf.fit(X_raz, y)

MultinomialNB()

In [21]:
raz_preds = clf.predict(X_raz_test)

In [22]:
print(classification_report(y_test, st_preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.96      0.89       936
         1.0       0.90      0.61      0.73       506

    accuracy                           0.84      1442
   macro avg       0.86      0.79      0.81      1442
weighted avg       0.85      0.84      0.83      1442



In [23]:
print(classification_report(y_test, raz_preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.81      0.97      0.88       936
         1.0       0.91      0.57      0.70       506

    accuracy                           0.83      1442
   macro avg       0.86      0.77      0.79      1442
weighted avg       0.84      0.83      0.82      1442



В итоге сравнения метрики вышли практически идентичные, с разницей всех итоговых видов F1 не более 2% (стоит отметить, что словари векторайзеров отличаются на 899 слова).

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [24]:
# ваш код

In [25]:
предложения = ["я и ты", "ты и я", "я, я и только я", "только не я", "он"]
я = [1, 1, 3, 1, 0]
ты = [1, 1, 0, 0, 0]
и = [1, 1, 1, 0, 0]
только = [0, 0, 1, 1, 0]
не = [0, 0, 0, 1, 0]
он = [0, 0, 0, 0, 1]

In [26]:
absf_df = pd.DataFrame.from_dict({"предложения":предложения, "я":я, "ты":ты, "и":и, "только":только, "не":не, "он":он})
absf_df.set_index("предложения", inplace=True)
absf_df = absf_df.astype(int)
absf_df

,я,ты,и,только,не,он
предложения,,,,,,
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [27]:
import numpy as np

new_ls = []

N = absf_df.shape[0]
for col in absf_df:
    new_col = []
    zeros = (absf_df[col] == 0).sum()
    for row in absf_df[col]:
        tf = int(row)
        idf = np.log(N / (N - zeros))
        new_col.append(tf * idf)
    new_ls.append(new_col)

tfidf_df = pd.DataFrame.from_dict({"предложения":предложения, "я":new_ls[0], "ты":new_ls[1], "и":new_ls[2], "только":new_ls[3], "не":new_ls[4], "он":new_ls[5]})
tfidf_df.set_index("предложения", inplace=True)
tfidf_df

,я,ты,и,только,не,он
предложения,,,,,,
я и ты,0.223144,0.916291,0.510826,0.000000,0.000000,0.000000
ты и я,0.223144,0.916291,0.510826,0.000000,0.000000,0.000000
"я, я и только я",0.669431,0.000000,0.510826,0.916291,0.000000,0.000000
только не я,0.223144,0.000000,0.000000,0.916291,1.609438,0.000000
он,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [28]:
data = pd.read_csv('labeled.csv')

In [29]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [30]:
train, test = train_test_split(data, test_size=0.2, shuffle=True)

train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [31]:
y_i = train.toxic.values
y_test_i = test.toxic.values

In [32]:
# clf1

In [33]:
vectorizer1 = CountVectorizer(encoding="utf-8", decode_error="ignore", strip_accents="unicode", lowercase=True, min_df=1)

In [34]:
X_1 = vectorizer1.fit_transform(train.comment)
X_1_test = vectorizer1.transform(test.comment) 

In [35]:
X_1.shape, X_1_test.shape

((11529, 59081), (2883, 59081))

In [54]:
clf1 = LogisticRegression(C=0.1, class_weight='balanced')
clf1.fit(X_1, y_i)
preds1 = clf1.predict(X_1_test)

In [37]:
probas1 = clf1.predict_proba(X_1_test)
probas1

array([[0.57425505, 0.42574495],
       [0.12413277, 0.87586723],
       [0.55077288, 0.44922712],
       ...,
       [0.08442178, 0.91557822],
       [0.65044956, 0.34955044],
       [0.79361829, 0.20638171]])

In [39]:
# clf2

In [40]:
vectorizer2 = TfidfVectorizer(encoding="utf-8", decode_error="ignore", lowercase=True,  strip_accents="unicode", min_df=1, max_df=1.0)

In [41]:
X_2 = vectorizer2.fit_transform(train.comment)
X_2_test = vectorizer2.transform(test.comment) 

In [42]:
X_2.shape, X_2_test.shape

((11529, 59081), (2883, 59081))

In [53]:
clf2 = RandomForestClassifier(n_estimators=110, max_depth=100, class_weight="balanced")
clf2.fit(X_2, y_i)
preds2 = clf2.predict(X_2_test)

In [44]:
probas2 = clf2.predict_proba(X_2_test)
probas2

array([[0.62949478, 0.37050522],
       [0.32581883, 0.67418117],
       [0.55048382, 0.44951618],
       ...,
       [0.50819747, 0.49180253],
       [0.81856485, 0.18143515],
       [0.87595999, 0.12404001]])

In [55]:
print("Метрики clf1 (LogReg, CountVec):\n")
print(classification_report(y_test_i, preds1))

Метрики clf1 (LogReg, CountVec):

              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88      1899
         1.0       0.73      0.85      0.79       984

    accuracy                           0.84      2883
   macro avg       0.82      0.84      0.83      2883
weighted avg       0.85      0.84      0.85      2883



In [56]:
print("Метрики clf2 (RanFor, TFIDFVec):\n")
print(classification_report(y_test_i, preds2, zero_division=0))

Метрики clf2 (RanFor, TFIDFVec):

              precision    recall  f1-score   support

         0.0       0.83      0.87      0.85      1899
         1.0       0.72      0.65      0.68       984

    accuracy                           0.79      2883
   macro avg       0.77      0.76      0.76      2883
weighted avg       0.79      0.79      0.79      2883



In [48]:
w1 = clf1.fit(X_1, y_i).coef_[0]
top_10_1 = []
for i in range(10):
    top = np.sort(w1)[::-1][i]
    top_index = list(w1).index(top)
    top_10_1.append(vectorizer1.get_feature_names_out()[top_index])
top_10_1

['хохлы',
 'ты',
 'хохлов',
 'быдло',
 'тебя',
 'сука',
 'русских',
 'блядь',
 'тебе',
 'тупые']

In [49]:
w2 = clf2.fit(X_2, y_i).feature_importances_
top_10_2 = []
for i in range(10):
    top = np.sort(w2)[::-1][i]
    top_index = list(w2).index(top)
    top_10_2.append(vectorizer1.get_feature_names_out()[top_index])
top_10_2

['ты', 'но', 'не', 'тебя', 'можно', 'на', 'то', 'если', 'очень', 'по']

Большая часть текстов, выделенных моделью логистической регрессии, действительно токсичная, но есть и недочеты. Так, как токсичные были выделены простые местоимения "ты", "тебя", "тебе". 
Модель random forests плохо справилась с задачей 

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [50]:
# ваш код

In [51]:
w1 = clf1.fit(X_1, y_i).coef_[0]
top_5_1 = []
for i in range(5):
    top = np.sort(w1)[::-1][i]
    top_index = list(w1).index(top)
    top_5_1.append(vectorizer1.get_feature_names_out()[top_index])
top_5_1

['хохлы', 'ты', 'хохлов', 'быдло', 'тебя']

In [52]:
w2 = clf2.fit(X_2, y_i).feature_importances_
top_5_2 = []
for i in range(5):
    top = np.sort(w2)[::-1][i]
    top_index = list(w2).index(top)
    top_5_2.append(vectorizer1.get_feature_names_out()[top_index])
top_5_2

['ты', 'не', 'можно', 'но', 'тебя']